# Intro to Deep Learning
<table align="left"><td>
  <a target="_blank"  href="https://github.com/Clemson-AI/Intro/blob/master/Intro_to_Deep_Learning_CAI.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on github
  </a>
</td><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/Clemson-AI/Intro/blob/master/Intro_to_Deep_Learning_CAI.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
</td></table>

In [1]:
import torch
import torchvision.models as models
from torchvision import transforms as T
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
from google.colab import files

# Tensors
![Tensor](https://miro.medium.com/max/1050/0*jGB1CGQ9HdeUwlgB)

In [ ]:
# Rank 0
torch.tensor(.6)

In [ ]:
# Rank 1 
torch.tensor([.7, 1.4, 2.1])

In [ ]:
# Rank 2
torch.randn(2,2)

tensor([[-0.5733, -0.4501],
        [ 0.7236,  0.9845]])

In [ ]:
# Rank 4
torch.randn(2,2,2,2)

tensor([[[[-0.3028, -0.4098],
          [ 0.4015,  0.2455]],

         [[ 1.6076,  0.5838],
          [ 0.6200, -0.6811]]],


        [[[ 0.3436,  1.0570],
          [ 0.3909,  1.9183]],

         [[ 0.3407,  0.4702],
          [-0.7572, -0.5975]]]])

# Matrix Multiplication

In [26]:
t1 = torch.tensor([5.0, 6.0])
t2 = torch.tensor([[5.0, 6.0], [6.0, 5.0], [.2, .7]])
t3 = torch.tensor([])

In [27]:
result = t1 * t2

tensor([[25.0000, 36.0000],
        [30.0000, 30.0000],
        [ 1.0000,  4.2000]])

# Activation Function

\begin{equation}

Sigmoid(x) = \sigma(x) = \dfrac{1}{1 + \exp(-x)}

\end{equation}

​	
![Sigmoid](https://pytorch.org/docs/stable/_images/Sigmoid.png)
\begin{equation}
ReLU(x) = x^+ = max(0,x)
\end{equation}

![ReLu](https://pytorch.org/docs/stable/_images/ReLU.png)


Sigmoid

In [ ]:
# Rank 0
r0data = torch.tensor(.6)
# Rank 1 
r1data = torch.tensor([.7, 1.4, 2.1])

In [ ]:
sig = nn.Sigmoid()

In [ ]:
sig(r0data)

tensor(0.6457)

In [ ]:
sig(r1data)

tensor([0.6682, 0.8022, 0.8909])

ReLU

In [ ]:
relu = nn.ReLU()

In [ ]:
relu(r1data)

tensor([0.7000, 1.4000, 2.1000])

In [ ]:
relu(data)

tensor(0.5000)

# Neural Network

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


# Loss

### <center>L1Loss</center>
\begin{equation}
loss = \dfrac{\sum_{i=1}^{n}∣y_i−x_i∣}{n}
\end{equation}

### <center>Binary Cross Entropy Loss</center>
<p align="center">
  <img src="https://miro.medium.com/max/1096/1*rdBw0E-My8Gu3f_BOB6GMA.png" />
</p>

In [ ]:
data = torch.tensor([5.0,5.0,5.0])
truth = torch.tensor([7.0,8.0,9.0])

In [28]:
# L1 Loss, mean absolute error (MAE) useful for Regression tasks
loss = nn.L1Loss(reduction='mean')

In [ ]:
loss(data, truth)

tensor(3.)

In [ ]:
# BCELoss is useful for Classification tasks
criterion = nn.BCELoss()

# Optimizer

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

# Transfer Learning with VGG Backbone
[Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)

In [ ]:
# vgg16 is a Convolutional Neural Network(CNN) trained on Imagenet 2014 - 1000 categories and 1.2 million images
vgg16 = models.vgg16(pretrained=True)

# Turn off training for vgg16
for param in vgg16.parameters():
    param.requires_grad = False

In [ ]:
vgg16

# Train

# Saving and Loading

# Action

In [ ]:
files.upload()